In [1]:
import boto3
from botocore.config import Config

region_name = "us-east-1"
#llm_model = "anthropic.claude-3-sonnet-20240229-v1:0"
llm_model = "anthropic.claude-3-haiku-20240307-v1:0"

def converse_with_bedrock(sys_prompt, usr_prompt):
    temperature = 0.1
    top_p = 0.3
    top_k = 10
    inference_config = {"temperature": temperature, "topP": top_p}
    additional_model_fields = {"top_k": top_k}
    response = boto3_client.converse(
        modelId=llm_model, 
        messages=usr_prompt, 
        system=sys_prompt,
        inferenceConfig=inference_config,
        additionalModelRequestFields=additional_model_fields
    )
    return response['output']['message']['content'][0]['text']

def init_boto3_client(region: str):
    retry_config = Config(
        region_name=region,
        retries={"max_attempts": 10, "mode": "standard"}
    )
    return boto3.client("bedrock-runtime", region_name=region, config=retry_config)


def create_prompt(sys_template, user_template, **kwargs):
    sys_prompt = [{"text": sys_template.format(**kwargs)}]
    usr_prompt = [{"role": "user", "content": [{"text": user_template.format(**kwargs)}]}]
    return sys_prompt, usr_prompt

boto3_client = init_boto3_client(region_name)


In [2]:
from py2neo import Graph
import os

os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "password"

graph = Graph()

In [3]:
def select_subgraph_dev(question, graph):
    question = question
    query = """
        MATCH (n:Title {level: "1"})
        RETURN n.value, id(n) as node_id
    """
    results = graph.run(query)
    subgraph_list = [(record["n.value"], record["node_id"]) for record in results]
    subgraph_list_with_number = [f"{i}. {subgraph[0]}" for i, subgraph in enumerate(subgraph_list)]

    sys_prompt_template = "당신은 AWS 매뉴얼 문서에 정통한 전문 엔지니어입니다.\n사용자의 질문에 가장 적절한 매뉴얼 문서 이름을 선택하는 것이 당신의 임무입니다. 관련된 문서가 없으면 빈 목록("")을 제공합니다."
    usr_prompt_template = "주어진 질문에 가장 관련성 높은 문서 이름 1개를 선택해주세요.\n#질문: {question}\n\n #문서 목록:\n {subgraph_list_with_number}\n\n #응답 형식: 선택된 문서의 인덱스 번호만 제공 (서두는 생략)"
    sys_prompt, usr_prompt = create_prompt(sys_prompt_template, usr_prompt_template, question=question, subgraph_list_with_number=subgraph_list_with_number)
    
    selected_id = converse_with_bedrock(sys_prompt, usr_prompt)
    try:
        if selected_id == "":
            return [], "", "generate_answer"

        else: 
            selected_subgraph_id = subgraph_list[int(selected_id)][1]
            print("Selected:", subgraph_list[int(selected_id)][0])
            return [selected_subgraph_id], subgraph_list[int(selected_id)][0], "traverse_child"
    except:
        return [], "", "generate_answer"

In [4]:
# 질문의 주제 선택
question = "Bedrock에서 제공하는 모델 목록"
target_node, subgraph, next_step = select_subgraph_dev(question, graph)
print(target_node, "|", subgraph, "|", next_step)

Selected: Amazon Bedrock
[0] | Amazon Bedrock | traverse_child


In [5]:
csv_list_response_format = "Your response should be a list of comma separated values, eg: `foo, bar` or `foo,bar`"

def traverse_child_dev(question, subgraph, graph, target_node):
    question = question
    node_id = target_node[0]
    child_list = []
    query = """
        MATCH (n)
        WHERE id(n) = $node_id
        OPTIONAL MATCH (n)-[:HAS_CHILD]->(c)
        RETURN n.value as node_value, c.value, id(c) as child_id
    """
    params = {"node_id": node_id}
    results = graph.run(query, params)

    node_value = None
    child_list = []

    for record in results:
        if node_value is None:
            node_value = record["node_value"]
        if record["c.value"] is not None:
            child_list.append((record["c.value"], record["child_id"]))
    
    print(f"Traversing '{node_value}'...")

    if not child_list:
        print("No child. Proceed to 'get_contents'...")
        return node_id, [], "get_contents"

    child_list_with_number = [f"{i}. {child}" for i, child in enumerate(child_list)]
    sys_prompt_template = """
    당신은 AWS 매뉴얼 문서에 정통한 전문 엔지니어입니다.
    당신의 임무는 사용자의 질문에 답변하기 위해, <{subgraph}> 매뉴얼 문서에서 참고할 하위 메뉴를 선택하는 것입니다.
    
    작업 순서:
    1. 주어진 하위 메뉴 목록 중 질문과 연관된 정보가 있는지 확인합니다. 직접적으로 연관된 메뉴가 없다면 반드시 빈 목록("")으로 응답합니다.
    2. 연관된 하위 메뉴가 있다면, 연관성이 가장 높은 메뉴의 인덱스 번호(0부터 시작)로 응답합니다.
     - 질문에서 요구하는 키워드와 일치하고, 특정 주제를 다루는 메뉴 항목에 높은 우선순위를 부여하세요.
     - 예를 들어, 일반적인 'Getting started' 가이드보다는 특정 기능이나 서비스에 대한 상세 설명이 있는 항목을 선호합니다. 
    3. 참고해야 할 문서가 1개 이상인 경우에 응답 형식에 맞춰 인덱스 번호의 목록으로 제공하세요.
    """
    usr_prompt_template = """
    #질문: {question}
     
    #메뉴 목록:
    {child_list_with_number}
     
    #응답 형식: {csv_list_response_format}. 
    """
    sys_prompt, usr_prompt = create_prompt(sys_prompt_template, usr_prompt_template, subgraph=subgraph, question=question, child_list_with_number=child_list_with_number, csv_list_response_format=csv_list_response_format)
    selected_ids = converse_with_bedrock(sys_prompt, usr_prompt)
    try:
        if selected_ids == '""' or selected_ids.strip() == "":
            return node_id, [], "get_contents"
        else:
            selected_id_list = [int(id.strip()) for id in selected_ids.split(',') if id.strip().isdigit()]
            selected_childs = [child_list[id][1] for id in selected_id_list] if selected_id_list else []
            return node_id, selected_childs, "traverse_child"
    except:
        return node_id, [], "get_contents"

In [6]:
# 질문의 주제 선택
question = "Bedrock의 Custom Model 활용 방법"
target_node, subgraph, next_step = select_subgraph_dev(question, graph)
print(target_node, "|", subgraph, "|", next_step)

# 적합한 헤더 찾기
while target_node != []:
    print("target_node:", target_node)
    cur_pos, target_node, next_step = traverse_child_dev(question, subgraph, graph, target_node)


Selected: Amazon Bedrock
[0] | Amazon Bedrock | traverse_child
target_node: [0]
Traversing 'Amazon Bedrock'...
target_node: [2071]
Traversing 'Custom models'...
target_node: [2111, 2218, 2186]
Traversing 'Submit a model customization job'...
No child. Proceed to 'get_contents'...


In [7]:
def get_contents_dev(question, cur_pos, order_pos):
    question = question
    node_id = cur_pos
    order_pos = order_pos

    count_query = """
        MATCH (n)-[:HAS_CONTENTS]->(c)
        WHERE id(n) = $node_id
        RETURN count(c) as document_count
    """
    params = {"node_id": node_id}
    count_result = graph.run(count_query, params).data()[0]
    document_count = count_result['document_count']
    print(f"Num Documents: {document_count}")

    if document_count <= 10:
        content_query = """
            MATCH (n)-[:HAS_CONTENTS]->(c)
            WHERE id(n) = $node_id AND c.order >= $order_pos
            RETURN c.order, c.text
            ORDER BY c.order
            LIMIT 5
        """
        
        params = {"node_id": node_id, "order_pos": order_pos}
        content_results = graph.run(content_query, params)
        contents = [record["c.text"] for record in content_results]
        if document_count > 5:
            optional_prompt1 ="\n- 정보가 질문에 답변하기에 적합하나 정보가 중단되어 다음 페이지 조회가 필요한 경우, `Partial`로 응답합니다.\n"
            optional_prompt2 = "`Partial`, "
        else:
            optional_prompt1 = ""
            optional_prompt2 = ""
        context = " ".join(contents)

        sys_prompt_template = """
        당신은 AWS에 정통한 전문 엔지니어입니다. 당신의 임무는 주어진 질문을 해결하기 위해 수집된 사전 정보가 적합한지 판단하는 것입니다.
        
        작업 순서:
        - 수집된 정보를 활용해 주어진 질문을 해결할 수 있을지 판단합니다. 정보가 질문을 해결하기에 적합하지 않으면 `None`으로 응답합니다.{partial1}
        - 충분한 정보가 수집되어 답변하기에 충분하다면 `Complete`으로 응답합니다.
        """
        usr_prompt_template = "서두는 생략하고, `None`, {partial2}`Complete` 중 하나로 답변하세요 #수집된 정보: {context}\n\n #질문:\n {question}"

        sys_prompt, usr_prompt = create_prompt(sys_prompt_template, usr_prompt_template, partial1=optional_prompt1, partial2=optional_prompt2, question=question, context=context)
        status = converse_with_bedrock(sys_prompt, usr_prompt)
    else:
        status = "Search Requested."

    return status, context

In [8]:
# 질문의 주제 선택
question = "Bedrock의 Agent 기능에서 대화 memory 기능을 활용하는 방법"
target_node, subgraph, next_step = select_subgraph_dev(question, graph)
print(target_node, "|", subgraph, "|", next_step)

# 적합한 헤더 탐색
while target_node != []:
    print("target_node:", target_node)
    cur_pos, target_node, next_step = traverse_child_dev(question, subgraph, graph, target_node)

# 문서 얻어내기
order_pos = 0
status, context = get_contents_dev(question, cur_pos, order_pos)
print("Status:", status)
print("==============")
print(context)
print("==============")

Selected: Amazon Bedrock
[0] | Amazon Bedrock | traverse_child
target_node: [0]
Traversing 'Amazon Bedrock'...
target_node: [1493]
Traversing 'Agents for Amazon Bedrock'...
target_node: [1594]
Traversing 'Use memory to retain conversational context across multiple sessions'...
target_node: [1599]
Traversing 'Configure memory for your Amazon Bedrock agent'...
No child. Proceed to 'get_contents'...
Num Documents: 3
Status: Complete
To configure memory for your agent, you must first enable memory and then optionally specify
[the retention period for the memory. You can enable memory for your agent when you create or](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-create.html)
[update your agent.](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-manage.html#agents-edit)  
To learn how to configure memory for your agent, select the tab corresponding to your method of
choice and follow steps.  
Console  
To configure memory for your agent  
1. If you're not already in

In [9]:
# 질문의 주제 선택 (찾을 수 없는 정보)
question = "Bedrock의 가격 정책"
target_node, subgraph, next_step = select_subgraph_dev(question, graph)
print(target_node, "|", subgraph, "|", next_step)

# 적합한 헤더 찾기
while target_node != []:
    cur_pos, target_node, next_step = traverse_child_dev(question, subgraph, graph, target_node)
    print(target_node, "|", subgraph, "|", next_step)

# 문서 얻어내기
order_pos = 0
status, context = get_contents_dev(question, cur_pos, order_pos)
print("Status:", status)
print("==============")
print(context)
print("==============")

Selected: Amazon Bedrock
[0] | Amazon Bedrock | traverse_child
Traversing 'Amazon Bedrock'...
[3552, 2246] | Amazon Bedrock | traverse_child
Traversing 'Quotas for Amazon Bedrock'...
[3583, 3559] | Amazon Bedrock | traverse_child
Traversing 'Provisioned Throughput quotas'...
No child. Proceed to 'get_contents'...
[] | Amazon Bedrock | get_contents
Num Documents: 1
Status: None
The following quotas apply to Provisioned Throughput.  
Note  
If a quota is marked as not adjustable through Service Quotas, you can submit a request
[through the limit increase form to be considered for an increase.](https://console.aws.amazon.com/support/home#/case/create?issueType=service-limit-increase)  
|Description|Default|Adjustable through Service Quotas|
|---|---|---|
|Model units that can be distributed across no-commit ment Provisioned Throughpu ts|2|No|
|Model units that can be distributed across Provision ed Throughputs with commitment|0|No|  
Provisioned Throughput quotas 1592


In [11]:
from langchain_aws import ChatBedrock
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

def generate_answer_dev(question, context):
    # Prompt setting
    sys_prompt_template = "당신은 AWS에 정통한 전문 엔지니어입니다. 주어진 사전 정보만 활용하여, 사용자 질문에 답변을 생성하세요. 사전 정보로 주어지지 않은 내용에 대한 질문에는 모른다고 답변하세요."
    usr_prompt_template = "#사전 정보: {context}\n\n #사용자 질문:\n {question}"
    prompt = ChatPromptTemplate.from_messages([("system", sys_prompt_template), ("human",usr_prompt_template)])

    # Model setting
    model_kwargs = {
            "temperature": 0.5,
            "max_tokens": 4096
        }
    llm = ChatBedrock(model_id="anthropic.claude-3-5-sonnet-20240620-v1:0", region_name="us-west-2", model_kwargs=model_kwargs, streaming=True)   

    # Output setting
    parser = StrOutputParser()

    # Chain
    chain = prompt | llm | parser
    for chunk in chain.stream({"context": context, "question": question}):
        print(chunk, end="", flush=True)

In [12]:
# 질문의 주제 선택
question = "Bedrock에서 Agent 성능을 최적화 하는 방법"
target_node, subgraph, next_step = select_subgraph_dev(question, graph)
print(target_node, "|", subgraph, "|", next_step)

# 적합한 헤더 찾기
while target_node != []:
    print("target_node:", target_node)
    cur_pos, target_node, next_step = traverse_child_dev(question, subgraph, graph, target_node)

# 문서 얻어내기
order_pos = 0
status, context = get_contents_dev(question, cur_pos, order_pos)

print("Status:", status)
generate_answer_dev(question, context)



Selected: Amazon Bedrock
[0] | Amazon Bedrock | traverse_child
target_node: [0]
Current Node: Amazon Bedrock
6
target_node: [1493]
Current Node: Agents for Amazon Bedrock
4, 13
target_node: [1718, 1594]
Current Node: Customize an Amazon Bedrock agent
2
target_node: [1859]
Current Node: Optimize performance for Amazon Bedrock agents
0
target_node: [1861]
Current Node: Optimize performance for Amazon Bedrock agents using a single knowledge base
-Child not found-
Number of documents: 7
Agents for Amazon Bedrock offers options to choose different flows that can optimize on latency
for simpler use cases in which agents have a single knowledge base. To ensure that your agent
is able to take advantage of this optimization, check that the following conditions apply to the
relevant version of your agent:  
-  Your agent contains only one knowledge base.  
-  Your agent contains no action groups or they are all disabled.  
-  Your agent doesn't request more information from the user if it doesn'

In [13]:
# 질문의 주제 선택
question = "Bedrock에서 Knowledge Base를 테스트하는 방법"
target_node, subgraph, next_step = select_subgraph_dev(question, graph)
print(target_node, "|", subgraph, "|", next_step)

# 적합한 헤더 찾기
while target_node != []:
    print("target_node:", target_node)
    cur_pos, target_node, next_step = traverse_child_dev(question, subgraph, graph, target_node)

# 문서 얻어내기
order_pos = 0
status, context = get_contents_dev(question, cur_pos, order_pos)

print(context)

# 문서 찾기
order_pos = 0
status, context = get_contents_dev(question, cur_pos, order_pos)

generate_answer_dev(question, context)



Selected: Amazon Bedrock
[0] | Amazon Bedrock | traverse_child
target_node: [0]
Current Node: Amazon Bedrock
15
target_node: [1209]
Current Node: Knowledge bases for Amazon Bedrock
10
target_node: [1415]
Current Node: Test a knowledge base in Amazon Bedrock
0
target_node: [1417]
Current Node: Query the knowledge base and return results or generate responses
-Child not found-
Number of documents: 7
To learn how to query your knowledge base, select the tab corresponding to your method of choice
and follow the steps.  
Console  
To test your knowledge base  
1. Sign in to the AWS Management Console using an IAM role with Amazon Bedrock
[permissions, and open the Amazon Bedrock console at https://console.aws.amazon.com/](https://console.aws.amazon.com/bedrock/)
[bedrock/.](https://console.aws.amazon.com/bedrock/)  
2. From the left navigation pane, select Knowledge bases.  
3. In the Knowledge bases section, do one of the following actions:  
-  Choose the radio button next to the knowledg

In [14]:
# 질문의 주제 선택
question = "Bedrock의 가격 정책"
target_node, subgraph, next_step = select_subgraph_dev(question, graph)
print(target_node, "|", subgraph, "|", next_step)

# 적합한 헤더 찾기
while target_node != []:
    print("target_node:", target_node)
    cur_pos, target_node, next_step = traverse_child_dev(question, subgraph, graph, target_node)

# 문서 얻어내기
order_pos = 0
status, context = get_contents_dev(question, cur_pos, order_pos)

print(context)

# 문서 찾기
order_pos = 0
status, context = get_contents_dev(question, cur_pos, order_pos)

generate_answer_dev(question, context)



Selected: Amazon Bedrock
[0] | Amazon Bedrock | traverse_child
target_node: [0]
Current Node: Amazon Bedrock
0, 26
target_node: [3552, 2246]
Current Node: Quotas for Amazon Bedrock
2, 9
target_node: [3583, 3559]
Current Node: Provisioned Throughput quotas
-Child not found-
Number of documents: 1
The following quotas apply to Provisioned Throughput.  
Note  
If a quota is marked as not adjustable through Service Quotas, you can submit a request
[through the limit increase form to be considered for an increase.](https://console.aws.amazon.com/support/home#/case/create?issueType=service-limit-increase)  
|Description|Default|Adjustable through Service Quotas|
|---|---|---|
|Model units that can be distributed across no-commit ment Provisioned Throughpu ts|2|No|
|Model units that can be distributed across Provision ed Throughputs with commitment|0|No|  
Provisioned Throughput quotas 1592
Number of documents: 1
죄송합니다만, 주어진 사전 정보에는 Bedrock의 전반적인 가격 정책에 대한 내용이 포함되어 있지 않습니다. 사전 정보는 Provisioned

In [ ]:
def get_sibling_contents_dev(cur_pos, order_pos):
    
    return

